In [9]:
import pandas as pd

import numpy as np
import math as m


class SearchNode:
    #Defining our class for nodes to be checked
    

    def __init__(self, x, y):
        
        #Initializing 
        
        self.start = 0
        self.solution = False
        self.type = None
        self.in_path = False

        self.x = x
        self.y = y

        self.children = []
        self.parent = None

        self.cost = 0
        self.g = float('inf')  # g(s)
        self.h = None  # h(s)

    def f(self):
        #Getting the total cost, i.e. cost so far + estimated to goal
        
        return self.g + self.h

    def __str__(self):
        #Found this neat way to print results
        return "x: " + str(self.x) + ", y: " + str(self.y) + ", cost: " + str(self.cost)


# Found various sources for the implementation. Mostly from "Essentials of the A* Algorithm" handout

def aStar():
    #The A* algorithm, returning True if a path from start to finish has been found
   

    closed_nodes = []
    open_nodes = [start]

    while True:

        current_node = open_nodes.pop(0)
        closed_nodes.append(current_node)

        if current_node.solution:
            return True  # Solution found

        generateSuccessors(current_node)

        for child in current_node.children:
            
            if child not in open_nodes and child not in closed_nodes:
                attachEval(child, current_node)
                open_nodes.append(child)
                open_nodes.sort(key=lambda x: x.f())  # Sorting after lowest estimated cost 

            elif current_node.g + child.cost < child.g:  # If a cheaper path is found
                attachEval(child, current_node)
                if child in closed_nodes:
                    propagateImprovements(child)


def generateSuccessors(current_node):
    #Need to add all neighbour nodes to the nodes children.
    
    
    for node in all_nodes:
        if node.cost < float('inf'):  # Check if node is not a wall
            if (current_node.y == node.y) and (current_node.x == node.x - 1 or current_node.x == node.x + 1):
                current_node.children.append(node)
                
            elif (current_node.x == node.x) and (current_node.y == node.y - 1 or current_node.y == node.y + 1):
                current_node.children.append(node)


def attachEval(child, parent):
    #Calculating the cost at this point and estimating cost to go. Void func
    
    child.parent = parent
    child.g = parent.g + child.cost

    #Setting h
    dist = euclidean(child)
    child.h = dist * (child.cost / weight)  # (Euclid distance) * (Node cost / Average cost)


def propagateImprovements(parent):
    #Need to check if our path is optimal, if not, change path and update costs etc.
   
    for child in parent.children:
        if child.g > parent.g + child.cost:
            child.parent = parent
            child.g = parent.g + child.cost
            propagateImprovements(child)


def createNodes():  # e.g. "board1.txt"
    #Need to gather information from our files so we can run algorithm


    nodes = []
    game_board = []
    start_node = None
    goal_node = None
    file = open(filename, "r")
    for x_coor, line in enumerate(file.readlines()):
        game_board.append([None] * len(line.rstrip()))
        for y_coor, node in enumerate(line.rstrip()):
            game_board[x_coor][y_coor] = node

            if node == '#':  # Wall
                wall_node = SearchNode(x_coor, y_coor)
                wall_node.cost = float('inf')
                wall_node.type = node
                nodes.append(wall_node)

            elif node == 'A':  # Start node
                start_node = SearchNode(x_coor, y_coor)
                start_node.start = True
                start_node.type = node
                nodes.append(start_node)

            elif node == 'B':  # Goal node
                goal_node = SearchNode(x_coor, y_coor)
                goal_node.solution = True
                goal_node.type = node
                nodes.append(goal_node)

            else:  # Path node
                search_node = SearchNode(x_coor, y_coor)
                search_node.cost = cost_dict[node]
                search_node.type = node
                nodes.append(search_node)

    return start_node, goal_node, nodes, game_board



def visualize():
    #Visualizing by text for part 1. I know it seems a bit bulky...
    length = len(board[0]) 
    count = 0
    visualize=""
    for node in all_nodes:
        if node.start:
            visualize+='S'
            count+=1
            continue
        elif node.solution:
            visualize+='F'
            count+=1
            continue
        if node.in_path:
            visualize+='-'
        elif node.cost == float('inf'):
            visualize+='#'
        else:
            visualize+='0'
        
        count+=1
        if count==length:
            visualize+='\n'
            count=0
    print(visualize, "\n the char - denotes the path, # denotes obstacle, S start and F finish. 0 is various other possible path elements")


def visualize2():
    #Visualizing by text for part 2. I know it seems a bit bulky...
    length = len(board[0]) 
    count = 0
    visualize=""
    for node in all_nodes:
        if node.start:
            visualize+='S'
            count+=1
            continue
        elif node.solution:
            visualize+='F'
            count+=1
            continue
        if node.in_path:
            visualize+='-'
        elif 1 <= node.cost <= 15:
            visualize+='#'
        else:
            visualize+='0'
        count+=1
        if count==length:
            visualize+='\n'
            count=0
    print(visualize, "\n the char - denotes the path, # denotes obstacle, S start and F finish. 0 is various other possible path elements")



def weightCalc():
    #Need to find the average cost of all our nodes
    
    weight = 0
    counter = 0
    for node in all_nodes:
        if not node.start and not node.solution:  # Path node
            weight += node.cost
            counter += 1

    return weight / counter


def euclidean(node):
    #Basic
   
    return np.sqrt(m.pow(goal.x - node.x, 2) + m.pow(goal.y - node.y, 2))


# Testing

filename = "board1part2.txt"  # Choose filename, our filenames are: board1.txt, board1part2.txt, board2.txt, board2part2.txt etc...
print("Part 1 or part 2? Enter as integers: ")
part = int(input())

cost_dict = {"w": 100, "m": 50, "f": 10, "g": 5, "r": 1, ".": 1}


start, goal, all_nodes, board = createNodes()

# Setting initial states
start.h = euclidean(start)
start.g = 0
goal.h = 0

weight = weightCalc()

if aStar():
    print("Path found")
    print("Start-node:", start)
    print("Goal-node:", goal)

    node = goal.parent
    while node != start:
        node.in_path = True
        print(node)
        node = node.parent

    if part == 1:
        visualize()
    if part == 2:
        visualize2()

        

Part 1 or part 2? Enter as integers: 
2
Path found
Start-node: x: 0, y: 17, cost: 0
Goal-node: x: 9, y: 17, cost: 0
x: 8, y: 17, cost: 5
x: 8, y: 18, cost: 5
x: 8, y: 19, cost: 5
x: 8, y: 20, cost: 5
x: 8, y: 21, cost: 5
x: 8, y: 22, cost: 5
x: 8, y: 23, cost: 5
x: 7, y: 23, cost: 10
x: 6, y: 23, cost: 10
x: 5, y: 23, cost: 1
x: 5, y: 24, cost: 1
x: 5, y: 25, cost: 1
x: 5, y: 26, cost: 1
x: 5, y: 27, cost: 1
x: 5, y: 28, cost: 1
x: 5, y: 29, cost: 1
x: 4, y: 29, cost: 1
x: 3, y: 29, cost: 1
x: 2, y: 29, cost: 1
x: 1, y: 29, cost: 1
x: 1, y: 28, cost: 1
x: 1, y: 27, cost: 1
x: 1, y: 26, cost: 1
x: 1, y: 25, cost: 1
x: 1, y: 24, cost: 1
x: 1, y: 23, cost: 1
x: 1, y: 22, cost: 1
x: 1, y: 21, cost: 1
x: 1, y: 20, cost: 1
x: 1, y: 19, cost: 1
x: 1, y: 18, cost: 1
x: 1, y: 17, cost: 1
00000############S#################00000
000##############-------------######0000
00###########################-######0000
00#############00000#########-#######000
0#############0000000########-######0000
00###